드라이브 연결 + git clone + 필요 패키지 설치

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%shell
git clone https://github.com/DataCampusKoreaUniv2022/DataCampusKU2022.git

In [ ]:
%%shell
cd /content/DataCampusKU2022/DINO_model
pip install -r requirements.txt

In [ ]:
%%shell
cd /content/DataCampusKU2022/DINO_model/models/dino/ops
python setup.py build install

NO CUDA RUNTIME error 발생 시 

런타임 - 하드웨어 가속기 : GPU 인지 확인



---



# 시각화

### **중요**

모듈 설치 후 아래 셀 실행 시 No module named 'MultiScaleDeformableAttention' error 발생할 경우

**상단 '런타임' - '런타임 다시 시작' 후 아래 셀부터 다시 컴파일**

In [ ]:
%cd /content/DataCampusKU2022/DINO_model/

import os, sys
import torch, json
import numpy as np

from main import build_model_main
from util.slconfig import SLConfig
from datasets import build_dataset
from util.demo_visualizer import COCOVisualizer
from util import box_ops

# 0. Model 로드하기

체크포인트 파일 다운로드

In [ ]:
!wget https://github.com/DataCampusKoreaUniv2022/DataCampusKU2022/releases/download/v1.0.0/checkpoint_best_regular.pth

In [ ]:
model_config_path = "/content/DataCampusKU2022/DINO_model/config/DINO/DINO_4scale.py" # path of the model config file
model_checkpoint_path = "/content/DataCampusKU2022/DINO_model/checkpoint_best_regular.pth" # path of the model checkpoint

In [ ]:
args = SLConfig.fromfile(model_config_path) 
args.device = 'cuda' 
model, criterion, postprocessors = build_model_main(args)
checkpoint = torch.load(model_checkpoint_path, map_location='cpu')
model.load_state_dict(checkpoint['model'])
_ = model.eval()

In [ ]:
# load coco names
with open('/content/DataCampusKU2022/DINO_model/util/20class_plant_coco_id2name.json') as f: # id-class matching json file
    id2name = json.load(f)
    id2name = {int(k):v for k,v in id2name.items()}

# 1. 이미지 시각화

## 1.1 데이터셋 불러오기

In [ ]:
!wget https://github.com/DataCampusKoreaUniv2022/DataCampusKU2022/releases/download/v1.0.0/dino_data.zip

In [ ]:
!unzip dino_data.zip

In [ ]:
args.dataset_file = 'coco'
args.coco_path = "/content/DataCampusKU2022/DINO_model/dino_data" # the path of coco
args.fix_size = False

dataset_val = build_dataset(image_set='val', args=args)

## 1.2 test 이미지 annotation 시각화하기

In [ ]:
import random
ranNum = random.randrange(2000)
print(ranNum)
image, targets = dataset_val[ranNum]

test 이미지 중 하나를 가져옵니다.

In [ ]:
box_label = [id2name[int(item)] for item in targets['labels']]
gt_dict = {
    'boxes': targets['boxes'],
    'image_id': targets['image_id'],
    'size': targets['size'],
    'box_label': box_label,
}
vslzr = COCOVisualizer()
vslzr.visualize(image, gt_dict, savedir=None)

In [ ]:
box_label

어떻게 annotate 되어있는지 확인합니다.

## 1.3 모델에 넣어서 시각화하기

In [ ]:
output = model.cuda()(image[None].cuda())
output = postprocessors['bbox'](output, torch.Tensor([[1.0, 1.0]]).cuda())[0]

In [ ]:
thershold = 0.2 # set a thershold

scores = output['scores']
labels = output['labels']
boxes = box_ops.box_xyxy_to_cxcywh(output['boxes'])
select_mask = scores > thershold

In [ ]:
box_label = [id2name[int(item)] for item in labels[select_mask]]
pred_dict = {
    'boxes': boxes[select_mask],
    'size': targets['size'],
    'box_label': box_label
}
vslzr.visualize(image, pred_dict, savedir=None)

In [ ]:
for i, bl in enumerate(box_label):
    print(bl, round(scores.tolist()[i], 4))

모델을 돌려서 나온 결과를 시각화합니다.

# 2. 커스텀 이미지 시각화하기

In [ ]:
from PIL import Image
import datasets.transforms as T

In [ ]:
image = Image.open("/content/DataCampusKU2022/DINO_model/dino_data/val2017/000000000000.jpg").convert("RGB") # 커스텀 이미지 path

open 함수 내의 파일 경로명을 커스텀 이미지 경로로 교체합니다. (default 값은 test 예시)

In [ ]:
transform = T.Compose([
    T.RandomResize([800], max_size=1333),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
image, _ = transform(image, None)

In [ ]:
output = model.cuda()(image[None].cuda())
output = postprocessors['bbox'](output, torch.Tensor([[1.0, 1.0]]).cuda())[0]

In [ ]:
# 결과물 시각화
thershold = 0.2 # thershold

vslzr = COCOVisualizer()

scores = output['scores']
labels = output['labels']
boxes = box_ops.box_xyxy_to_cxcywh(output['boxes'])
select_mask = scores > thershold

box_label = [id2name[int(item)] for item in labels[select_mask]]
pred_dict = {
    'boxes': boxes[select_mask],
    'size': torch.Tensor([image.shape[1], image.shape[2]]),
    'box_label': box_label
}
vslzr.visualize(image, pred_dict, savedir=None, dpi=100)

In [ ]:
for i, bl in enumerate(box_label):
    print(bl, round(scores.tolist()[i], 4))